In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Layer
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras import backend as K

class RBFLayer(Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = gamma

    def build(self, input_shape):
        # Create a trainable weight variable for the centers of the RBFs
        self.centers = self.add_weight(name='centers',
                                      shape=(self.units, input_shape[-1]),
                                      initializer='uniform',
                                      trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        # Calculate the radial basis functions for each input sample and each RBF center
        diff = K.expand_dims(inputs) - self.centers  # shape: (batch_size, units, input_dim)
        norm = K.sum(K.square(diff), axis=-1)  # shape: (batch_size, units)
        rbf = K.exp(-self.gamma * norm)  # shape: (batch_size, units)

        return rbf

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.units
    
url = 'https://raw.githubusercontent.com/ravsssh/HandballPrediction/main/Handball%20Prediction/datashotiqr/LB.csv'
df = pd.read_csv(url, delimiter=',')
# Menentukan input(X) dan output (y)
X = df.drop(columns=["SHOT","GOAL","7MSHOT","6MSHOT","9MSHOT","WINGSHOT","PIVOTSHOT","FASTBREAKSHOT"])
y = df.GOAL
# Membagi dataset 80:20 untuk train dan test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# normalisasi dataset agar tidak bias
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

validation_results_df = pd.DataFrame(columns=['R2_Test', 'MSE', 'MAE'])

for _ in range(20):
    model = Sequential()
    model.add(Flatten(input_shape=(6,)))
    model.add(RBFLayer(6, 0.5))
    model.add(Dense(1, activation='linear'))
    model.compile(optimizer='rmsprop', loss=mean_squared_error, metrics=['mean_absolute_error']) 
    history = model.fit(X_train, y_train, batch_size=8, epochs=100, validation_data=(X_test, y_test))
    
    pred_test = model.predict(X_test)
    
    r2_test = r2_score(y_test, pred_test)
    mse = history.history['val_loss'][-1]  # Last MSE from validation data
    mae = history.history['val_mean_absolute_error'][-1]  # Last MAE from validation data
    
    print(f"R-Squared Test Score: {r2_test}")
    
    # Append results to the validation results DataFrame
    validation_results_df = validation_results_df.append({'R2_Test': r2_test,
                                                          'MSE': mse,
                                                          'MAE': mae}, ignore_index=True)
    
    # Reset the TensorFlow session to clear the model
    tf.keras.backend.clear_session()

# Save the validation results DataFrame to a CSV file
validation_results_df.to_csv('validation_resultsacc.csv', index=False)


Epoch 1/100
310/310 [==============================] - 1s 2ms/step - loss: 10.3836 - mean_absolute_error: 2.4059 - val_loss: 6.4556 - val_mean_absolute_error: 1.8092
Epoch 2/100
310/310 [==============================] - 0s 1ms/step - loss: 5.2312 - mean_absolute_error: 1.6504 - val_loss: 4.1256 - val_mean_absolute_error: 1.5838
Epoch 3/100
310/310 [==============================] - 0s 1ms/step - loss: 3.9503 - mean_absolute_error: 1.5108 - val_loss: 3.3483 - val_mean_absolute_error: 1.4430
Epoch 4/100
310/310 [==============================] - 0s 1ms/step - loss: 3.4013 - mean_absolute_error: 1.4092 - val_loss: 2.9806 - val_mean_absolute_error: 1.3523
Epoch 5/100
310/310 [==============================] - 0s 1ms/step - loss: 3.0827 - mean_absolute_error: 1.3291 - val_loss: 2.6814 - val_mean_absolute_error: 1.2928
Epoch 6/100
310/310 [==============================] - 0s 1ms/step - loss: 2.8189 - mean_absolute_error: 1.2715 - val_loss: 2.4401 - val_mean_absolute_error: 1.2316
Epoch 7/1

C:\Users\Wandi\AppData\Local\Temp\ipykernel_144824\2581008955.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


310/310 [==============================] - 1s 2ms/step - loss: 11.4438 - mean_absolute_error: 2.5125 - val_loss: 6.6794 - val_mean_absolute_error: 1.8571
Epoch 2/100
310/310 [==============================] - 0s 1ms/step - loss: 5.1768 - mean_absolute_error: 1.6551 - val_loss: 3.7930 - val_mean_absolute_error: 1.5207
Epoch 3/100
310/310 [==============================] - 0s 2ms/step - loss: 3.5576 - mean_absolute_error: 1.4261 - val_loss: 2.8712 - val_mean_absolute_error: 1.3295
Epoch 4/100
310/310 [==============================] - 0s 1ms/step - loss: 2.9767 - mean_absolute_error: 1.3052 - val_loss: 2.5527 - val_mean_absolute_error: 1.2417
Epoch 5/100
310/310 [==============================] - 0s 1ms/step - loss: 2.7065 - mean_absolute_error: 1.2321 - val_loss: 2.3226 - val_mean_absolute_error: 1.1929
Epoch 6/100
310/310 [==============================] - 0s 1ms/step - loss: 2.4962 - mean_absolute_error: 1.1821 - val_loss: 2.1413 - val_mean_absolute_error: 1.1322
Epoch 7/100
310/310 [

C:\Users\Wandi\AppData\Local\Temp\ipykernel_144824\2581008955.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


310/310 [==============================] - 1s 2ms/step - loss: 12.6709 - mean_absolute_error: 2.7975 - val_loss: 7.8206 - val_mean_absolute_error: 1.9482
Epoch 2/100
310/310 [==============================] - 0s 1ms/step - loss: 5.9622 - mean_absolute_error: 1.7161 - val_loss: 4.5758 - val_mean_absolute_error: 1.6351
Epoch 3/100
310/310 [==============================] - 0s 1ms/step - loss: 4.1418 - mean_absolute_error: 1.5347 - val_loss: 3.4413 - val_mean_absolute_error: 1.4772
Epoch 4/100
310/310 [==============================] - 0s 1ms/step - loss: 3.4372 - mean_absolute_error: 1.4268 - val_loss: 3.0063 - val_mean_absolute_error: 1.3752
Epoch 5/100
310/310 [==============================] - 0s 1ms/step - loss: 3.0920 - mean_absolute_error: 1.3520 - val_loss: 2.7345 - val_mean_absolute_error: 1.2941
Epoch 6/100
310/310 [==============================] - 0s 1ms/step - loss: 2.8231 - mean_absolute_error: 1.2737 - val_loss: 2.4757 - val_mean_absolute_error: 1.2525
Epoch 7/100
310/310 [

...

C:\Users\Wandi\AppData\Local\Temp\ipykernel_144824\2581008955.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


310/310 [==============================] - 1s 2ms/step - loss: 9.9100 - mean_absolute_error: 2.2377 - val_loss: 6.5893 - val_mean_absolute_error: 1.9251
Epoch 2/100
310/310 [==============================] - 0s 1ms/step - loss: 5.2077 - mean_absolute_error: 1.7140 - val_loss: 4.0137 - val_mean_absolute_error: 1.6011
Epoch 3/100
310/310 [==============================] - 0s 1ms/step - loss: 3.6391 - mean_absolute_error: 1.4821 - val_loss: 3.0568 - val_mean_absolute_error: 1.4031
Epoch 4/100
310/310 [==============================] - 0s 1ms/step - loss: 3.0679 - mean_absolute_error: 1.3624 - val_loss: 2.7147 - val_mean_absolute_error: 1.3097
Epoch 5/100
310/310 [==============================] - 0s 1ms/step - loss: 2.7869 - mean_absolute_error: 1.2899 - val_loss: 2.4591 - val_mean_absolute_error: 1.2539
Epoch 6/100
310/310 [==============================] - 0s 1ms/step - loss: 2.5619 - mean_absolute_error: 1.2357 - val_loss: 2.2588 - val_mean_absolute_error: 1.1949
Epoch 7/100
310/310 [=

C:\Users\Wandi\AppData\Local\Temp\ipykernel_144824\2581008955.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,


310/310 [==============================] - 1s 2ms/step - loss: 6.4472 - mean_absolute_error: 2.0784 - val_loss: 5.5453 - val_mean_absolute_error: 1.9468
Epoch 2/100
310/310 [==============================] - 0s 1ms/step - loss: 5.0863 - mean_absolute_error: 1.7864 - val_loss: 4.2852 - val_mean_absolute_error: 1.6809
Epoch 3/100
310/310 [==============================] - 0s 1ms/step - loss: 4.0469 - mean_absolute_error: 1.5775 - val_loss: 3.4865 - val_mean_absolute_error: 1.4967
Epoch 4/100
310/310 [==============================] - 0s 1ms/step - loss: 3.4504 - mean_absolute_error: 1.4482 - val_loss: 3.0260 - val_mean_absolute_error: 1.3962
Epoch 5/100
310/310 [==============================] - 0s 1ms/step - loss: 3.0722 - mean_absolute_error: 1.3665 - val_loss: 2.6807 - val_mean_absolute_error: 1.3039
Epoch 6/100
310/310 [==============================] - 0s 1ms/step - loss: 2.7649 - mean_absolute_error: 1.2858 - val_loss: 2.3885 - val_mean_absolute_error: 1.2339
Epoch 7/100
310/310 [=

C:\Users\Wandi\AppData\Local\Temp\ipykernel_144824\2581008955.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  validation_results_df = validation_results_df.append({'R2_Test': r2_test,
